In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
from tqdm import tqdm

In [4]:
tqdm.pandas()

In [85]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [86]:
sentences = train["question_text"].progress_apply(lambda x: x.split()).values

100%|██████████| 1306122/1306122 [00:06<00:00, 207709.33it/s]


In [87]:
vocab = build_vocab(sentences)

100%|██████████| 1306122/1306122 [00:03<00:00, 391708.51it/s]


In [8]:
from gensim.models import KeyedVectors

In [9]:
glove_path = 'data/glove.840B.300d/glove.840B.300d.txt'

In [21]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [23]:
# glove2word2vec(glove_path,"data/glove_word2vec_format.txt")

In [6]:
!pip install gensim
import gensim

     |████████████████████████████████| 24.1 MB 3.6 MB/s eta 0:00:01


/opt/conda/lib/python3.8/site-packages/azure/storage/blob/_deserialization.py:641: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif num_empty_lines is 2:
/opt/conda/lib/python3.8/site-packages/azure/storage/blob/_serialization.py:189: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if requests is None or len(requests) is 0:
/opt/conda/lib/python3.8/site-packages/azure/storage/blob/_serialization.py:301: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(serialized_query) is not 0:
/opt/conda/lib/python3.8/site-packages/azure/storage/blob/_upload_chunking.py:409: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if n is 0 or self._buffer.closed:
/opt/conda/lib/python3.8/site-packages/azure/storage/blob/baseblobservice.py:1071: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if lease_duration is not -1 and \
/opt/conda/lib/python3.8/site-packages/azure/storage/blob/baseblobservice.py:2779: SyntaxWarning: "is not" with a literal. 

In [8]:
from gensim.models import KeyedVectors

In [13]:
glove_vectors = KeyedVectors.load_word2vec_format("data/glove_word2vec_format.txt",binary=False,no_header=False)

In [84]:
glove_vectors.most_similar("due",topn=20)

[('caused', 0.7013909220695496),
 ('owing', 0.6679489612579346),
 ('result', 0.6542508006095886),
 ('despite', 0.6377608776092529),
 ('affected', 0.6364753842353821),
 ('cause', 0.6272486448287964),
 ('because', 0.6262850165367126),
 ('severe', 0.6229299306869507),
 ('causing', 0.6226324439048767),
 ('likely', 0.6223006844520569),
 ('lack', 0.621307909488678),
 ('resulting', 0.6160615086555481),
 ('Due', 0.6126062273979187),
 ('occur', 0.6097978949546814),
 ('delayed', 0.6067006587982178),
 ('however', 0.6018208861351013),
 ('significant', 0.6002013087272644),
 ('after', 0.5976076722145081),
 ('problems', 0.5904189348220825),
 ('reduced', 0.5902970433235168)]

In [88]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [90]:
oov = check_coverage(vocab,glove_vectors)

100%|██████████| 508823/508823 [00:00<00:00, 539226.27it/s]

Found embeddings for 33.16% of vocab
Found embeddings for  88.16% of all text


In [92]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [93]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
sentences = train["question_text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

100%|██████████| 1306122/1306122 [00:03<00:00, 421032.82it/s]


In [94]:
oov = check_coverage(vocab,glove_vectors)

100%|██████████| 253623/253623 [00:00<00:00, 513828.75it/s]

Found embeddings for 71.38% of vocab
Found embeddings for  99.44% of all text


In [97]:
len(vocab)

253623